# ScanBot

The AI-driven bot that will read all your scanned documents and extract information into a database automatically!

## Exercise

Follow this notebook and add missing code sections denoted by "TODO".
To get a high-level understanding of the ScanBot, jump to the **Main** section.

## Preparation

In [1]:
%pip install -r requirements.txt

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/48/62/63419a90b502aa5a590c83ffbd30f0a93fe15aeb63d0374e898e612f4f03/openai-1.30.5-py3-none-any.whl.metadata
  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/58/d9/796181a30827b12101786c21301f0f4536597a9249530916b1fdb5bbad91/openpyxl-3.1.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for regex from https://files.pythonhosted.org/packages/8b/ee/05f14a99a81f1a897a9146f3f565efb116ad6412f875f52e895c02666825/regex-2024.5.15-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
  Obtaining dependency inf

## Imports

In [1]:
import os
import openai
import yaml
import pandas as pd
import base64
import re
import shutil
import time

## Insert your API Key

In [2]:
openai.api_key = "sk-proj-iHrdMRQrydnSDtQUIzU6T3BlbkFJCoAngxToaT2????????"

## Helper Functions

In [60]:
# Function to load prompts from files
def load_prompt(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()


# Function to analyze the document and extract meta data
def analyze_document(file_path, prompt):
    # create image data url
    with open(file_path, "rb") as file:
        image_url = 'TODO...' # TODO! Hint: How can you send an image via an API?

    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                # TODO! Hint: Use ChatGPT playground or Docs
            ],
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        # hard coded sleep time to avoid rate limit issues
        time.sleep(15)
        return analyze_document(file_path, prompt)


# Function to generate a filename based on meta data
def generate_filename(meta_data):
    # <doc type>_<date>_<title>.jpg
    # if any of the fields are missing, use "unknown"
    string = ""
    string += str(meta_data.get("document_type", "unknown")) + "_"
    string += str(meta_data.get("author", "unknown")) + "_"
    string += str(meta_data.get("date", "unknown")) + "_"
    string += str(meta_data.get("title", "unknown"))
    # Make sure that the filename is a valid filename
    string = re.sub(r"[^\w\s-]", "", string)
    return string + ".jpg"


# Function to save meta data to a YAML file
def save_meta_file(meta_data, meta_file_path):
    # Save json file as yaml
    with open(meta_file_path, "w", encoding="utf-8") as file:
        yaml.dump(meta_data, file, default_flow_style=False,
                  allow_unicode=True)


# Function to save meta data to CSV and Excel files
def save_meta_data_to_csv_excel(meta_data_list, csv_path, excel_path):
    # check if the csv file exists
    if not os.path.exists(csv_path):
        existing_df = pd.DataFrame()
    else:
        existing_df = pd.read_csv(csv_path)

    new_df = pd.DataFrame(meta_data_list)
    merged_df = pd.concat([existing_df, new_df]).drop_duplicates(subset='original_filepath')
    merged_df.to_csv(csv_path, index=False, encoding="utf-8")
    merged_df.to_excel(excel_path, index=False)


def parse_meta_data(response):
    # Parse the YAML string from the response
    yaml_string = "TODO" # TODO get the yaml content between "```yaml" and "```"
    meta_info = "TODO" # TODO use regular expression to remove \n characters
    # When the YAML string is in the correct format, it can be loaded as a JSON object
    meta_json = yaml.safe_load(meta_info)
    return meta_json


def save_meta_data(meta_json, file_path):
    # Generate new file name
    new_filename = generate_filename(meta_json)
    # Rename the document file
    renamed_file_location = os.path.join('database', 'renamed files', new_filename)
    shutil.copy2(file_path, renamed_file_location)
    # Create meta data file
    meta_file_path = os.path.join('database', 'meta files', new_filename.split('.')[0] + ".yaml")
    save_meta_file(meta_json, meta_file_path)
    # Add new file name to meta data
    meta_json["original_filepath"] = file_path
    meta_json["renamed_filename"] = new_filename
    return new_filename

def get_original_filenames(csv_path):
    # Extract all "original_filepath" from the meta_data csv file
    original_filepaths = []
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        original_filepaths = df["original_filepath"].tolist()
    return original_filepaths

## Main

Main Script for Document Analysis and Meta Data Generation

This script performs the following steps:
1. Load the scanned document from the specified directory.
2. Analyze the document using the OpenAI GPT-4o model.
3. Generate meta data for the document including type, date, author, recipient,
   title, keywords, summary, entities, language, and page count.
4. Save the meta data as a YAML file.
5. Generate an informative file name based on the meta data.
6. Rename the document file with the newly generated informative name.
7. Finally, create a csv and excel file with all the meta data.

Project Structure:
- database/
  - meta files/
  - renamed files/
- documents/
  - <scanned_documents>.jpg
- prompts/
  - analyze_document.txt
  - generate_title.txt
- .env
- .gitignore
- main.py
- README.md

Instructions:
1. Place your scanned/uploaded documents in the 'documents' directory.
2. Modify the prompt in 'prompts/analyze_document.txt' according to your requirements.
3. Run all cells in the jupyter notebook

**Running the next cell will delete previous extractions from the `database` directory**

In [50]:
# Delete everything inside the database folder in one line
shutil.rmtree('database', ignore_errors=True)

# First, make sure that database/meta files and database/renamed_files exist
os.makedirs('database/meta files')
os.makedirs('database/renamed files')

Let's checkout the prompt first:

In [66]:
prompt_path = "prompts/analyze_document.txt"
prompt = load_prompt(prompt_path)
print(prompt)

Please analyze the attached scan of a document and extract the relevant information. Create a metadata file in YAML format that includes only the following fields and nothing else:

- document_type: The type or category of the document (e.g., invoice, receipt, contract, letter).
- date: The date of the document, if available (e.g., invoice date, letter date).
- author: The person or organization that created or sent the document.
- recipient: The person or organization to whom the document is addressed.
- title: The main title or subject of the document.
- keywords: Important terms or phrases extracted from the document to facilitate searching.
- summary: A brief summary of the content of the document.
- entities: Important entities mentioned in the document (e.g., names, places, organizations).
- page: The page number of the document (e.g., page 1 of 3).

The response must be strictly in YAML format and contain only the metadata fields specified above, without any additional text or e

Use ChatGPT-4o to collect information about all scanned documents; subsequently collect all data in a table

In [64]:
meta_data_list = []
document_dir = "documents"
csv_path = "database/meta_data.csv"
excel_path = "database/meta_data.xlsx"
# Extract all "original_filepath" from the meta_data csv file
original_filepaths = get_original_filenames(csv_path)

for filename in os.listdir(document_dir):
    # We want to extract the information from each scanned document in the documents folder
    extensions = (".jpg", ".jpeg", ".png")
    if (filename.endswith(extensions) and
        (os.path.join(document_dir, filename) not in original_filepaths)):
        file_path = os.path.join(document_dir, filename)
        print(f"Processing {filename}...")
        # Analyze the document to get meta data
        llm_response = analyze_document(file_path, prompt) # TODO: Implement this function
        # Parse the meta data from the response
        meta_json = parse_meta_data(llm_response) # TODO: Implement this function
        # Save the meta data and rename the document file
        new_file_name = save_meta_data(meta_json, file_path)
        meta_data_list.append(meta_json)
        print(f"Processed {filename}, renamed it to {new_file_name}")

# Save all meta data to CSV and Excel files
save_meta_data_to_csv_excel(meta_data_list, csv_path, excel_path)
print(f"Meta data saved to {csv_path} and {excel_path}")

Meta data saved to database/meta_data.csv and database/meta_data.xlsx


Now put your own scans into the `documents` folder. Did ChatGPT extracted the contents correctly? 